# 🧪 Synthetic Test Data Generator
*Generate realistic synthetic datasets for testing and development purposes*

In [ ]:
# Imports
import os
import openai
import gradio as gr
from dotenv import load_dotenv
from datetime import datetime
import json
import csv
import io

# Load API key securely from .env file
load_dotenv(override=True)
openai.api_key = os.getenv("OPENAI_API_KEY")
if openai.api_key:
    print(f"OpenAI API Key exists and begins {openai.api_key[:8]}")
else:
    print("OpenAI API Key not set")

# Global constants
DEFAULT_OUTPUT_FORMAT = "json"  # or "csv"
DEFAULT_MODEL = "gpt-4o"
DEFAULT_COUNT = 10

In [ ]:
# System prompt

system_prompt = """
You are a Synthetic Dataset Generator AI.
Your job is to generate realistic-looking synthetic data for testing purposes, based on a domain and a free-text description.
Always return ONLY the raw content, without explanations, markdown formatting, or code block markers like ```.
NEVER skip or truncate — return exactly the number of records requested.
If variation is relevant to the domain (e.g. different names, cities, prices, categories), make sure the values are diverse.

Format rules:
---
If the user asks for JSON:
Return an array of objects only. For example:
[
  {"title": "Product A", "price": 19.99, "in_stock": true},
  {"title": "Product B", "price": 22.5, "in_stock": false}
]

If the user asks for CSV:
Return plain text with comma-separated values. For example:
title,price,in_stock
Product A,19.99,true
Product B,22.5,false
---

Avoid any form of commentary or explanation. Output only the raw data.
"""

In [ ]:
# Helper functions

def generate_user_prompt(domain, description, output_format, count):
    """
    Creates a detailed user prompt for OpenAI to generate synthetic data.
    
    This function builds a structured prompt that tells the AI exactly what kind
    of data to generate, in what format, and how many records.

    Args:
        domain (str): The business or data domain (e.g., 'ecommerce', 'healthcare')
        description (str): Detailed description of desired data fields and structure
        output_format (str): Desired format ('json' or 'csv')
        count (int): Number of records to generate

    Returns:
        str: Formatted prompt string optimized for AI data generation
    """
    
    user_prompt = f"""
        Generate synthetic records with the following settings:
        Domain: {domain}
        Description: {description}
        Number of records: {count}
        Data format: {output_format.upper()}
        
        Return ONLY raw data. 
        No explanations, no markdown formatting, no comments. 
        Strictly complete records.
        If possible, include diverse values in fields where variation is meaningful (e.g. names, categories, prices).
        """
    return user_prompt
    

def call_openai(system_prompt, user_prompt, model=DEFAULT_MODEL):
    """
    Call the OpenAI API and return the raw model output.
    
    This function handles the actual API communication with OpenAI's chat completion
    endpoint. Uses a low temperature for consistent output and sets token limits
    to prevent truncation issues.

    Args:
        system_prompt (str): System-level instructions for the AI
        user_prompt (str): User-specific prompt with generation details
        model (str): OpenAI model name to use

    Returns:
        str: Raw content generated by the model, stripped of whitespace
        
    Raises:
        openai.APIError: If the API call fails
        openai.RateLimitError: If rate limits are exceeded
    """
    
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.1,  # Low temperature for consistent, focused output
        max_tokens=4000   # Hopefully sufficient for ~100 records, prevents truncation

    )
    return response.choices[0].message.content.strip()

def validate_inputs(domain, description, output_format, count):
    """
    Validate user inputs before processing to provide better error messages.
    
    Args:
        domain (str): Data domain
        description (str): Data description
        output_format (str): Output format
        count (int): Record count
        
    Raises:
        ValueError: If any input is invalid
    """
    
    if not domain.strip():
        raise ValueError("Domain cannot be empty")
    if not description.strip():
        raise ValueError("Description cannot be empty")
    if count <= 0:
        raise ValueError("Record count must be greater than 0")


def validate_output(content, output_format, count):
    """
    Validate that the returned data matches expected format and record count.
    
    This function performs strict validation to ensure the AI generated exactly
    what was requested. Different validation logic for JSON vs CSV formats.

    Args:
        content (str): The raw generated data from OpenAI
        output_format (str): Expected format ('json' or 'csv')
        count (int): Expected number of records

    Returns:
        bool: True if validation passes
        
    Raises:
        ValueError: If data format is invalid or record count doesn't match
        json.JSONDecodeError: If JSON is malformed
    """
    
    if output_format == "json":
        # Parse JSON and verify it's a list with correct length
        parsed = json.loads(content)
        if not isinstance(parsed, list):
            raise ValueError("JSON output must be an array of objects")
        if len(parsed) != count:
            raise ValueError(f"Expected {count} records, got {len(parsed)}")
    
    elif output_format == "csv":
        # Parse CSV lines and verify structure
        lines = [line for line in content.splitlines() if line.strip()]
        if len(lines) < 2:
            raise ValueError("CSV must have header and at least one data row")
        
        # Check record count (excluding header)
        data_rows = lines[1:]
        if len(data_rows) != count:
            raise ValueError(f"Expected {count} data rows, got {len(data_rows)}")
    
    return True

def save_output_file(content, output_format):
    """
    Save generated data to a timestamped file in organized folder structure.
    
    Creates a 'synthetic_datasets' folder if it doesn't exist and saves the file
    with a timestamp to avoid naming conflicts. Ensures UTF-8 encoding for 
    international character support.

    Args:
        content (str): Raw generated content to save
        output_format (str): File extension to use ('json' or 'csv')

    Returns:
        str: Full file path of the saved file
    """
    
    # Generate timestamp for unique filenames
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"synthetic_data_{timestamp}.{output_format}"
    folder = "synthetic_datasets"
    
    # Create directory if it doesn't exist (exist_ok=True prevents errors if folder already exists)
    os.makedirs(folder, exist_ok=True)
    
    # Join folder and filename into full path (cross-platform compatible)
    filepath = os.path.join(folder, filename)

    # Write file with UTF-8 encoding for international character support
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(content)

    return filepath

In [ ]:
# Main function

def create_synthetic_data(domain, description, output_format, count):
    """
    Main orchestration function that handles the complete data generation workflow.
    
    This function coordinates all steps: input validation, prompt generation,
    AI API calls, output validation, and file saving. Includes error handling.

    Args:
        domain (str): Business domain for the synthetic data
        description (str): Detailed description of desired data structure
        output_format (str): Desired output format ('json' or 'csv')
        count (int): Number of records to generate

    Returns:
        tuple: (status_message: str, file_path: str or None)
            - status_message: Success message or error description
            - file_path: Path to generated file, or None if generation failed
    """
    
    try:
        # Validate inputs before processing
        validate_inputs(domain, description, output_format, count)
        
        # Generate AI prompt and call OpenAI API
        user_prompt = generate_user_prompt(domain, description, output_format, count)
        generated_data = call_openai(system_prompt, user_prompt)
        
        # Validate the generated output
        validate_output(generated_data, output_format, count)
        
        # Save to file and return success
        file_path = save_output_file(generated_data, output_format)
        filename = os.path.basename(file_path)  # Show only filename, not full path
        
        return f"✅ Dataset generated successfully!\n📁 File: {filename}\n📊 Records: {count}", file_path
        
    except json.JSONDecodeError as e:
        return f"❌ JSON format error: {str(e)}\nTry reducing record count or simplifying field descriptions.", None
    
    except ValueError as e:
        return f"❌ Validation error: {str(e)}", None
    
    except Exception as e:
        return f"❌ Unexpected error: {str(e)}\nPlease try again or contact support.", None

In [ ]:
# Gradio UI

with gr.Blocks() as ui:
    gr.Markdown("## 🧪 Synthetic Test Data Generator")
    gr.Markdown("Generate realistic synthetic datasets for testing and development purposes")

    with gr.Row():
        with gr.Column():
            domain_input = gr.Textbox(
                label="Domain", 
                placeholder="e.g., ecommerce, healthcare, finance, education",
                info="The business or industry context for your data"
            )
            
            description_input = gr.Textbox(
                label="Data Description", 
                lines=4, 
                placeholder="Describe the fields you want: names, prices, dates, categories, etc.\nExample: 'Customer data with name, email, age, city, purchase_amount, and signup_date'",
                info="Be specific about field names and data types you need"
            )
        
        with gr.Column():
            output_format = gr.Radio(
                choices=["json", "csv"],
                value=DEFAULT_OUTPUT_FORMAT, 
                label="Output Format",
                info="JSON for API testing, CSV for spreadsheet analysis"
            )
            
            count_input = gr.Dropdown(
                choices=[10, 25, 50], 
                value=DEFAULT_COUNT, 
                label="Number of Records",
                info="Higher counts may take longer to generate"
            )

    submit = gr.Button("🚀 Generate Dataset", variant="primary")
    
    with gr.Row():
        status_output = gr.Textbox(label="Generation Status", lines=3)
        download_file = gr.File(label="Download Generated Dataset")

    submit.click(
        fn=create_synthetic_data,
        inputs=[domain_input, description_input, output_format, count_input],
        outputs=[status_output, download_file]
    )

ui.launch(share=False, inbrowser=True)